<a href="https://colab.research.google.com/github/tapesh-santra/MediaBiasAnalysis/blob/master/Copy_of_news_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# First we shall train a model to identify unreliable news articles

In [0]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
# Load training and test data
project_dir='/content/drive/My Drive/news_crawler/fake_news_data/'
train=pd.read_csv(project_dir+'train.csv')
test=pd.read_csv(project_dir+'test.csv')

In [0]:
# Prepare data for feature engineering
test=test.fillna(' ')
train=train.fillna(' ')

# create n-gram features
count_vectorizer = CountVectorizer(ngram_range=(1, 2),min_df=0.01,max_df=0.7, stop_words='english')
counts = count_vectorizer.fit_transform(train['text'].values)

# IDF normalized term frequencies
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(counts)

#Extract feature from test data
test_counts = count_vectorizer.transform(test['text'].values)
test_tfidf = transformer.fit_transform(test_counts)

In [0]:
# Train a model
targets = train['label'].values

#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf, targets, random_state=0)

#Fit models


In [0]:

from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)
#Extra Tree Classifier
Extr = ExtraTreesClassifier(n_estimators=25,n_jobs=4)
Extr.fit(X_train, y_train)
print('Accuracy of ExtrTrees classifier on training set: {:.2f}'
     .format(Extr.score(X_train, y_train)))
print('Accuracy of Extratrees classifier on test set: {:.2f}'
     .format(Extr.score(X_test, y_test)))

Accuracy of ExtrTrees classifier on training set: 1.00
Accuracy of Extratrees classifier on test set: 0.92


In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gbc=GradientBoostingClassifier(learning_rate=0.1,n_estimators=50)
gbc.fit(X_train,y_train)
print('Accuracy of GradientBoosting classifier on training set: {:.2f}'
     .format(gbc.score(X_train, y_train)))
print('Accuracy of GradientBoosting classifier on test set: {:.2f}'
     .format(gbc.score(X_test, y_test)))


Accuracy of GradientBoosting classifier on training set: 0.92
Accuracy of GradientBoosting classifier on test set: 0.92


In [0]:
#Adaboost classifier
from sklearn.tree import DecisionTreeClassifier
Adab= AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=25)
Adab.fit(X_train, y_train)
print('Accuracy of Adaboost classifier on training set: {:.2f}'
     .format(Adab.score(X_train, y_train)))
print('Accuracy of Adaboost classifier on test set: {:.2f}'
     .format(Adab.score(X_test, y_test)))

Accuracy of Adaboost classifier on training set: 0.96
Accuracy of Adaboost classifier on test set: 0.93


In [0]:
Rando= RandomForestClassifier(n_estimators=25)

Rando.fit(X_train, y_train)
print('Accuracy of randomforest classifier on training set: {:.2f}'
     .format(Rando.score(X_train, y_train)))
print('Accuracy of randomforest classifier on test set: {:.2f}'
     .format(Rando.score(X_test, y_test)))

Accuracy of randomforest classifier on training set: 1.00
Accuracy of randomforest classifier on test set: 0.93


In [0]:
#Simple logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
print('Accuracy of Lasso classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Lasso classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Lasso classifier on training set: 1.00
Accuracy of Lasso classifier on test set: 0.94


In [0]:
#Save the model to the disk
import pickle
project_dir='/content/drive/My Drive/news_crawler/fake_news_data/'
pickle.dump(logreg,open(project_dir+'logreg.pkl','wb'))

# Time to scrape some data from "News" Channels
## Let's start with CNN
Install some useful libraries first 

In [0]:
!pip install beautifulsoup4


In [0]:
!pip3 install newspaper3k

In [0]:
!pip install feedparser

In [0]:
# Load the model
import pickle
clf=pickle.load(open(project_dir+'logreg.pkl','rb'))

In [0]:
#import feedparser as fp
import json
import newspaper
from newspaper import Article
from bs4 import BeautifulSoup
from time import mktime
from datetime import datetime
import urllib.request
import requests

In [0]:
#We shall download links from google news api (it's depricated but still works)
#First fox news
fox_news=[]
for page in range(1,6):
  url='https://newsapi.org/v2/everything?q=politics&sources=fox-news&page='+str(page)+'&sortBy=publishedAt&apiKey=f78d1be7057240008ba795b57929a5b9'
  data=json.loads(requests.get(url).text)
  if data['status']=='ok':
    for j in range(20):
      url=data['articles'][j]['url']
      article=Article(url)
      article.download()
      article.parse()
      fox_news.append(article.text)
      #print('...................................')
  else:
    print(data['message'])

In [0]:
#cnn news
cnn_news=[]
for page in range(1,6):
  url='https://newsapi.org/v2/everything?q=politics&sources=cnn&page='+str(page)+'&sortBy=publishedAt&apiKey=f78d1be7057240008ba795b57929a5b9'
  data=json.loads(requests.get(url).text)
  if data['status']=='ok':
    for j in range(20):
      url=data['articles'][j]['url']
      article=Article(url)
      article.download()
      article.parse()
      cnn_news.append(article.text)
      #print('...................................')
  else:
    print(data['message'])

In [0]:
#bbc news
bbc_news=[]
for page in range(1,6):
  url='https://newsapi.org/v2/everything?q=politics&sources=bbc-news&page='+str(page)+'&sortBy=publishedAt&apiKey=f78d1be7057240008ba795b57929a5b9'
  data=json.loads(requests.get(url).text)
  if data['status']=='ok':
    for j in range(20):
      url=data['articles'][j]['url']
      article=Article(url)
      article.download()
      article.parse()
      bbc_news.append(article.text)
      #print('...................................')
  else:
    print(data['message'])

In [0]:
#import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

#predict for cnn
#Extract feature from test data
cnn_counts = count_vectorizer.transform(np.array(cnn_news))
cnn_tfidf = transformer.fit_transform(cnn_counts)

#Now run the model on data
p_cnn=np.zeros((100,5))
p_cnn[:,0]=logreg.predict(cnn_tfidf)
p_cnn[:,1]=Extr.predict(cnn_tfidf)
p_cnn[:,2]=gbc.predict(cnn_tfidf)
p_cnn[:,3]=Adab.predict(cnn_tfidf)
p_cnn[:,4]=Rando.predict(cnn_tfidf)
cnn_reliability=np.sum(p_cnn,axis=1)


#######################
#fox news
fox_counts = count_vectorizer.transform(np.array(fox_news))
fox_tfidf = transformer.fit_transform(fox_counts)

#Now run the model on data
p_fox=np.zeros((100,5))
p_fox[:,0]=logreg.predict(fox_tfidf)
p_fox[:,1]=Extr.predict(fox_tfidf)
p_fox[:,2]=gbc.predict(fox_tfidf)
p_fox[:,3]=Adab.predict(fox_tfidf)
p_fox[:,4]=Rando.predict(fox_tfidf)
fox_reliability=np.sum(p_fox,axis=1)


#bbc news
bbc_counts = count_vectorizer.transform(np.array(bbc_news))
bbc_tfidf = transformer.fit_transform(bbc_counts)

#Now run the model on data
p_bbc=np.zeros((100,5))
p_bbc[:,0]=logreg.predict(bbc_tfidf)
p_bbc[:,1]=Extr.predict(bbc_tfidf)
p_bbc[:,2]=gbc.predict(bbc_tfidf)
p_bbc[:,3]=Adab.predict(bbc_tfidf)
p_bbc[:,4]=Rando.predict(bbc_tfidf)
bbc_reliability=np.sum(p_bbc,axis=1)

In [1]:
from collections import Counter
cnn=Counter(cnn_reliability)
fox=Counter(fox_reliability)
bbc=Counter(bbc_reliability)
#plt.figure();
#plt.bar([0,1,2,3,4,5],np.fliplr(cnn.values()))
#plt.bar([0,1,2,3,4,5],np.fliplr(fox.values()))
sorted_cnn = [cnn[k] for k in sorted(cnn)]
sorted_fox=  [fox[k] for k in sorted(fox)]
sorted_bbc=  [bbc[k] for k in sorted(bbc)]

ind = np.arange(6)  # the x locations for the groups
width = 0.28       # the width of the bars

fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(121)
rects1 = ax.bar(ind, sorted_cnn, width, color='royalblue',alpha=0.6)
rects2 = ax.bar(ind+width, sorted_fox, width, color='red',alpha=0.6)
rects3 = ax.bar(ind+2*width, sorted_bbc, width, color='seagreen',alpha=0.6)

# add some
ax.set_ylabel('Percentage of unreliable articles')
ax.set_xlabel('Number of models')
ax.set_title('Reliability of news articles')
ax.set_xticks(ind + width /2)
ax.set_xticklabels( ('0', '1', '2', '3', '4','5') )
#ax.set_alpha(0.9)
ax.legend( (rects1[0], rects2[0],rects3[0]), ('CNN', 'FOX','BBC') )

ax1=fig.add_subplot(122)
ax1.set_title('Percentage of articles found unreliable by at least 3 models')
rects11 = ax1.bar([0, 1, 2], [np.sum(sorted_cnn[3:]),0,0], 0.7, color='royalblue',alpha=0.6)
rects12 = ax1.bar([0, 1, 2], [0,np.sum(sorted_fox[3:]),0], 0.7, color='red',alpha=0.6)
rects13 = ax1.bar([0, 1, 2], [0,0,np.sum(sorted_bbc[3:])], 0.7, color='seagreen',alpha=0.6)
#ax1.legend( (rects11[0], rects12[0],rects13[0]), ('CNN', 'FOX','BBC') )
ax1.set_xticks(np.array([0,1,2]))
ax1.set_xticklabels( ('CNN', 'FOX', 'BBC') )
ax1.set_xlabel('News source')
ax1.set_ylabel('Percentage of unreliable articles')
plt.show()
plt.savefig(project_dir+'results.jpg',bbox_inches='tight')


NameError: ignored